In [14]:
# def vqgan load function and scripts
import torchvision, torch
import yaml
import sys
from omegaconf import OmegaConf
import einops
sys.path.append("/mnt/g/github/ZSE-SBIR/taming-transformers")
print(sys.path)
from taming.models.vqgan import VQModel, GumbelVQ

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None, is_gumbel=False):
  if is_gumbel:
    model = GumbelVQ(**config.model.params)
  else:
    model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()

['/mnt/g/github/ZSE-SBIR', '/home/bruce/anaconda3/lib/python310.zip', '/home/bruce/anaconda3/lib/python3.10', '/home/bruce/anaconda3/lib/python3.10/lib-dynload', '', '/home/bruce/anaconda3/lib/python3.10/site-packages', '/home/bruce/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg', '/home/bruce/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg', './', './', './taming', '/mnt/g/github/ZSE-SBIR/taming', '/mnt/g/github/ZSE-SBIR/taming-transformers', '/mnt/g/github/ZSE-SBIR/taming-transformers', '/mnt/g/github/ZSE-SBIR/taming-transformers']


In [11]:
# load vqgan model as model1024

config1024 = load_config("taming-transformers/logs/vqgan_imagenet_f16_1024/configs/model.yaml", display=False)
model1024 = load_vqgan(config1024, ckpt_path="taming-transformers/logs/vqgan_imagenet_f16_1024/checkpoints/last.ckpt").to('cuda')

print(model1024)
print(model1024, file=open("logs/model1024_info","w"))

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.
loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.
VQModel(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down): ModuleList(
      (0-1): 2 x Module(
        (block): ModuleList(
          (0-1): 2 x ResnetBlock(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
        )
        (attn): ModuleList()
        (downsample): Downsample(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
        )
      )
      (2): Module(
        (block): ModuleList(
          

In [16]:
import cv2 
# z = cv2.cvtColor(cv2.imread("./logs/patch_replace.jpg"),cv2.COLOR_BGR2RGB)
z = cv2.cvtColor(cv2.imread("./logs/patch_replace_32.jpg"),cv2.COLOR_BGR2RGB)
# z = cv2.cvtColor(cv2.imread("./logs/im_test29.jpg"),cv2.COLOR_BGR2RGB)

z = cv2.resize(z,(224,224))

print(z.shape, z.dtype)
# print(z)
z = torch.tensor(z, dtype=torch.float32)
# print(z)

z = einops.rearrange(z, "h w c -> c h w")
z = z.div(255.0)

print(z.shape)
torchvision.utils.save_image(z,"./logs/patch_replace_resize_32.jpg")

print(z.shape, z.dtype)
# cv2.imwrite("./logs/patch_replace_resize.jpg",z)
# torchvision.utils.save_image(z,"./logs/patch_replace_resize.jpg")
z = torch.unsqueeze(z, 0).cuda()
print(z.shape, z.dtype)


(224, 224, 3) uint8
torch.Size([3, 224, 224])
torch.Size([3, 224, 224]) torch.float32
torch.Size([1, 3, 224, 224]) torch.float32


In [17]:
# encode z

quant, _, _ = model1024.encode(z)

tensor([[[[ 0.7391, -0.6513,  1.2390,  ...,  0.8461,  0.8112,  0.3246],
          [ 0.7992, -0.7442,  0.8567,  ...,  0.5208,  0.2403,  0.5287],
          [ 0.6367, -0.0875,  0.6401,  ...,  0.5161,  0.7229,  0.2509],
          ...,
          [ 0.9427,  0.3736,  0.6635,  ...,  0.4156,  0.3345,  0.0542],
          [ 0.8933,  0.2916,  0.0831,  ...,  0.3609,  0.2841, -0.0250],
          [ 0.7820,  0.3972,  0.2835,  ...,  0.3362,  0.3411,  0.0597]],

         [[ 0.8107, -1.0841,  0.4812,  ...,  0.1374,  0.6985,  0.5401],
          [ 1.3672, -0.8527,  1.2638,  ..., -0.4616, -0.0102,  0.9422],
          [ 1.2879, -0.4392,  1.4836,  ..., -0.0880,  0.7862,  0.6526],
          ...,
          [ 1.2567, -0.1684,  0.7133,  ...,  0.5954,  0.3998,  0.1645],
          [ 1.1352,  1.1322,  0.7823,  ...,  0.6837,  0.6452,  0.2787],
          [ 0.8304,  0.8585,  1.0506,  ...,  0.6267,  0.6324,  0.3068]],

         [[ 0.6544, -0.7410,  1.1020,  ...,  0.6182,  1.1641,  0.2483],
          [ 0.9844, -0.9761, -

In [18]:

dec = model1024.decode(quant)
print(dec.shape)
torchvision.utils.save_image(dec,"logs/dec_patch_replaced_32.jpg")

torch.Size([1, 3, 224, 224])
